In [1]:
from hana_ml.dataframe import ConnectionContext
connection_context = ConnectionContext(address='lsvxc0103.sjc.sap.corp', port=30315, user='PAL_TEST',password='Init12345')

In [2]:
from data_load_utils import DataSets, Settings
from hana_ml.algorithms.pal.model_selection import GridSearchCV
from hana_ml.algorithms.pal.unified_classification import UnifiedClassification
Settings.load_config("../../config/e2edata.ini")
full_tbl, train_tbl, test_tbl, _ = DataSets.load_diabetes_data(connection_context)
diabetes_train = connection_context.table(train_tbl)
diabetes_test = connection_context.table(test_tbl)

uc_hgbdt = UnifiedClassification('HybridGradientBoostingTree')

gscv = GridSearchCV(estimator=uc_hgbdt, 
                    param_grid={'learning_rate': [0.1, 0.4, 0.7, 1],
                                'n_estimators': [4, 6, 8, 10],
                                'split_threshold': [0.1, 0.4, 0.7, 1]},
                    train_control=dict(fold_num=5,
                                       resampling_method='cv',
                                       random_state=1,
                                       ref_metric=['auc']),
                    scoring='error_rate')
gscv.fit(data=diabetes_train, key= 'ID',
         label='CLASS',
         partition_method='stratified',
         partition_random_state=1,
         stratified_column='CLASS',
         build_report=True)
features = diabetes_train.columns
features.remove('CLASS')
features.remove('ID')
pred_res = gscv.predict(diabetes_test, key='ID', features=features)

Table PIMA_INDIANS_DIABETES_TBL exists and data exists


In [3]:
from hana_ml.model_storage import ModelStorage

model_storage = ModelStorage(connection_context=connection_context)
gscv.estimator.name = 'HGBT' 
gscv.estimator.version = 1
model_storage.save_model(model=gscv.estimator)

In [4]:
from hana_ml.visualizers.unified_report import UnifiedReport

mymodel = model_storage.load_model('HGBT', 1)

UnifiedReport(mymodel).build().display(save_html='test1')

In [5]:
UnifiedReport(diabetes_test).build().display(save_html='test2')

Generating dataset report...: 100%|███████████████████████████████████| 4/4 [01:09<00:00, 17.28s/it]


In [6]:
#Simple assignment
from selenium import webdriver
from selenium.webdriver import Edge
from selenium.webdriver.common.by import By

In [8]:
#driver = Edge(executable_path=r'C:\Users\I319826\Downloads\msedgedriver.exe')  
driver = Edge(executable_path=r'C:\Users\I319826\Downloads\edgedriver_win32\msedgedriver.exe')  
driver.get("file:///C:/Users/i319826/Documents/Github/hanamlapi/nutest/testscripts/pal/notebooks/test1_unified_classification_model_report.html")

In [ ]:
#driver = Edge(executable_path=r'C:\Users\I544309\Downloads\edgedriver_win64\msedgedriver.exe')  
#driver.get("file:///C:/Users/I544309/Desktop/git/hanamlapi/nutest/testscripts/pal/notebooks/test2_dataset_report.html")

In [9]:
sidebar = driver.find_element_by_class_name('sidebar')
lst = sidebar.find_elements_by_class_name('nav-item')
try:
    lst[0].click()
except:
    driver.find_element_by_class_name('nav-link').click()
    time.sleep(0.1)
# if 'Dataset' in driver.title:
if len(lst) == 5:
    pagelst = ['overview_page', 'sample_page', 'variables_page', 'correlations_page', 'scatter_matrix_page']
# elif 'Model' in driver.title:
elif len(lst) == 6:
    pagelst = ['statistic_page', 'parameter_page', 'optimal_parameter_page', 'confusion_matrix_page',
               'variable_importance_page', 'metrics_page']
else:
    driver.quit()
    raise ValueError('Unsupport report type')
for i, p in enumerate(lst):
    try:
        p.click()
        style = driver.find_element(By.ID, pagelst[i]).get_attribute("style")
        if 'none' in style:
            driver.quit()
            raise ValueError('{} does not display properly'.format(pagelst[i]))
    except:
        driver.quit()
        raise ValueError('Cound not found {}'.format(pagelst[i]))
driver.quit()

In [10]:
driver.title

MaxRetryError: HTTPConnectionPool(host='localhost', port=52097): Max retries exceeded with url: /session/7a27869ad37518919807911d1190421c/title (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000253EF32DBB0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [ ]:
driver.quit()